In [1]:
import pickle
import shap
import os
import pandas as pd
from Fast_BEX import optimized_bex
from Bex import bex
import xgboost as xgb
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog

In [2]:
df = pd.read_csv("../CSVs and Output Files/HN_P2C_20231207.csv")
cwd = os.getcwd()

Columns (10,12,17,18,19,20,24) have mixed types. Specify dtype option on import or set low_memory=False.


In [4]:
df['Case'].unique()

array([  2,   4,   5,   6,   7,   8,  10,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       150, 152, 153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
       178, 179])

In [6]:
df = df[df['ScanContext'] == 'In Vivo']

# %% SNR and lifetime filtering
index = df[((df['snr_ch1'] < 30) & (df['Instrument'] == 'V4')) |
           ((df['snr_ch2'] < 30) & (df['Instrument'] == 'V4')) |
           ((df['snr_ch3'] < 30) & (df['Instrument'] == 'V4'))].index
df.drop(index, inplace=True)
index = df[((df['snr_ch1'] < 50) & (df['Instrument'] == 'FLImBrush')) |
           ((df['snr_ch2'] < 50) & (df['Instrument'] == 'FLImBrush')) |
           ((df['snr_ch3'] < 50) & (df['Instrument'] == 'FLImBrush'))].index
df.drop(index, inplace=True)

# R_data((R_data.lifet_avg_ch1 > 10 | R_data.lifet_avg_ch1 < 2),:) = [];
index = df[((df['lifet_avg_ch1'] > 16) | (df['lifet_avg_ch1'] < 1)) |
           ((df['lifet_avg_ch2'] > 16) | (df['lifet_avg_ch2'] < 1)) |
           ((df['lifet_avg_ch3'] > 16) | (df['lifet_avg_ch3'] < 1))].index
df.drop(index, inplace=True)

# %% For our current analysis we remove all V4 data and no channel 4 data
index = df[df['Instrument'] == 'FLImBrush'].index
df.drop(index, inplace=True)

In the following code, I changed (in line 3)
`selected_columns = df.columns[df.columns.str.contains(pattern)]` to
`selected_columns = df.columns[df.columns.str.match(pattern)]`

and same edits in line 28 - any neg effects?


In [34]:
# Extract columns based on specific patterns
pattern = r'(Case|Run|snr_|lifet_avg_|spec_int_|int_ratio_|Laguerre_coeffs_|Phasor_GH|Phasor_SH|WLI_)'
selected_columns = df.columns[df.columns.str.contains(pattern)]

# Add the "Label" column, which doesn't fit the patterns
selected_columns = selected_columns.insert(len(selected_columns), 'Label')

subset_df = df[selected_columns]
subset_df = subset_df.dropna()

subset_df = subset_df[subset_df['Label'] != 0]
# Replace values in the 'Label' column
subset_df['Label'] = subset_df['Label'].replace([1, 2, 18, 20, 25, 27, 29, 36, 37], 0)
subset_df['Label'] = subset_df['Label'].replace([5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 22, 24, 28, 30, 33], 1)

# Remove rows where 'Label' is not 0 or 1
subset_df = subset_df[subset_df['Label'].isin([0, 1])]

#bex_df = optimized_bex(subset_df) #excludes weak labels
bex_df = bex(subset_df)
f = 1

# Bex(subset_df, l_k=48, f_k1=6, f_k2=47)
# %% For our current analysis we remove all V4 data and no channel 4 data
from scipy.spatial.distance import pdist

pattern = r'(Case|Run|snr_|lifet_avg_|spec_int_|int_ratio_|Laguerre_coeffs_|WLI_)'
selected_columns = df.columns[df.columns.str.contains(pattern)]
selected_columns = selected_columns.insert(len(selected_columns), 'Label')
TrainTest_Data = bex_df[selected_columns]
TrainTest_Data.head()

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.


,Case,Run,lifet_avg_ch1,lifet_avg_ch2,lifet_avg_ch3,spec_int_ch1,spec_int_ch2,spec_int_ch3,int_ratio_ch1,int_ratio_ch2,...,Laguerre_coeffs_7_ch3,Laguerre_coeffs_8_ch3,Laguerre_coeffs_9_ch3,Laguerre_coeffs_10_ch3,Laguerre_coeffs_11_ch3,Laguerre_coeffs_12_ch3,WLI_frame_n,WLI_x_coord,WLI_y_coord,Label
0,100,5,6.781026,5.484078,5.954712,0.001646,0.006527,0.003781,0.137723,0.546027,...,0.035281,0.000746,0.008990,-0.001362,0.002206,-0.000534,20,679,245,1
1,100,5,5.849314,5.923007,6.376650,0.002385,0.007493,0.003722,0.175381,0.550946,...,0.034634,-0.015480,0.011322,-0.004491,0.002076,-0.000395,21,679,245,1
2,100,5,4.740269,5.756032,6.356454,0.003283,0.010584,0.006175,0.163797,0.528093,...,0.058853,-0.002603,0.016793,0.003447,0.003154,0.001915,22,678,246,1
3,100,5,4.675725,6.071339,7.349476,0.004329,0.012776,0.007327,0.177199,0.522911,...,0.042679,-0.029315,0.015650,-0.006797,0.002987,-0.000304,23,679,246,1
4,100,5,5.886445,5.379686,6.022543,0.004617,0.013433,0.007090,0.183663,0.534327,...,0.037973,-0.004290,0.015041,-0.001020,0.003769,0.000139,24,683,245,1


**.pkl File Selection**

In [7]:
def load_pickle_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(filetypes=[("Pickle files", "*.pkl")])
    
    if file_path:
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        root.destroy()  # Close the tkinter root window
        return data
    else:
        print("No file selected.")
        root.destroy()  # Close the tkinter root window
        return None
    
data = load_pickle_file()
xgb_model = data['model']
dataset = data['post_processing_data']
dataset.head(250)

2024-03-11 00:36:19.500 python[28821:12559356] +[CATransaction synchronize] called within transaction
2024-03-11 00:36:19.607 python[28821:12559356] +[CATransaction synchronize] called within transaction
2024-03-11 00:36:24.770 python[28821:12559356] +[CATransaction synchronize] called within transaction


,Case,Run,snr_ch1,snr_ch2,snr_ch3,WLI_frame_n,WLI_x_coord,WLI_y_coord,score,gt
2138,25,1,31.911923,34.841784,30.060749,9,685,474,0.598781,1
2139,25,1,31.524589,35.174167,31.011359,14,673,466,0.732532,1
2140,25,1,34.365631,37.667625,32.239650,15,674,467,0.498596,1
2141,25,1,35.723010,36.751284,30.918884,16,678,464,0.775350,1
2142,25,1,35.475074,35.965642,33.528947,18,676,436,0.822335,1
...,...,...,...,...,...,...,...,...,...,...
2383,25,1,31.933568,36.954090,32.422961,406,745,453,0.809451,1
2384,25,1,32.692458,36.680010,32.329930,407,745,446,0.985533,1
2385,25,1,34.332922,36.678309,32.246626,410,741,434,0.857804,1
2386,25,1,37.460354,40.248421,35.572587,425,730,368,0.895617,0


In [6]:
dataset['gt'].unique()

array([1, 0])

**SHAP Explainer**

In [9]:
#features = TrainTest_Data

explainer = shap.Explainer(xgb_model)
shap_values = explainer.shap_values(dataset)

[00:37:41] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


ValueError: This reshape error is often caused by passing a bad data matrix to SHAP. See https://github.com/shap/shap/issues/580.

**Prepare SHAP Explainer & Calculate SHAP Values**